In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import joblib

In [12]:
DATA_PATH = '../data/processed/procssed.csv'
OUTPUT_MODEL_PATH = '../output/models/model.joblib'
df = pd.read_csv(DATA_PATH)
df.describe()

,Engine_Size,Mileage,Negotiable,Car_Age,mileage_par_age,Make_Aston Martin,Make_Audi,Make_BMW,Make_BYD,Make_Bentley,...,Region_Jeddah,Region_Jubail,Region_Khobar,Region_Makkah,Region_Qassim,Region_Rare,Region_Riyadh,Region_Tabouk,Region_Taef,Price
count,8.035000e+03,8.035000e+03,8.035000e+03,8.035000e+03,8.035000e+03,8035.000000,8035.000000,8035.000000,8035.000000,8035.000000,...,8035.000000,8035.000000,8035.000000,8035.000000,8035.000000,8035.000000,8035.000000,8035.000000,8035.000000,8035.000000
mean,-7.781924e-17,3.466493e-16,-1.414895e-17,1.414895e-17,3.537238e-18,0.000373,0.006472,0.016428,0.000498,0.000871,...,0.131176,0.011325,0.011450,0.028251,0.038457,0.045302,0.402862,0.016179,0.017050,70720.712988
std,1.000062e+00,1.000062e+00,1.000062e+00,1.000062e+00,1.000062e+00,0.019320,0.080191,0.127123,0.022308,0.029505,...,0.337614,0.105823,0.106396,0.165701,0.192308,0.207978,0.490504,0.126172,0.129467,54088.861711
min,-1.507413e+00,-1.371595e+00,-6.771428e-01,-1.372674e+00,-1.779550e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2659.060000
25%,-8.485003e-01,-6.816147e-01,-6.771428e-01,-6.778515e-01,-6.315726e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,45000.000000
50%,-1.895880e-01,-2.236932e-01,-6.771428e-01,-3.304404e-01,-8.280287e-02,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,58000.000000
75%,7.328893e-01,4.905610e-01,1.476793e+00,3.643817e-01,5.189421e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,73750.000000
max,3.763886e+00,3.148166e+00,1.476793e+00,8.875952e+00,1.045521e+01,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,340000.000000


In [4]:
X = df.drop("Price", axis=1)
y = df["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
rf = RandomForestRegressor(random_state=42)
param_grid = {
        'n_estimators': [100, 200], 
        'max_features': [0.8, 1.0], 
        'min_samples_leaf': [1, 2]
} 
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                               cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
    
grid_search.fit(X_train, y_train)
best_rf_model = grid_search.best_estimator_

Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [10]:
import numpy as np

def evaluate_model(model, X_test, y_test):

    y_pred = model.predict(X_test)
    
    metrics = {
        'MSE': mean_squared_error(y_test, y_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R2': r2_score(y_test, y_pred),
    }
    
    return metrics, y_pred

def print_metrics(metrics):

    print("نتائج تقييم النموذج:")
    print(f"الدقة (MSE): {metrics['MSE']:.4f}")
    print(f"الدقة (rmse): {metrics['rmse']:.4f}")
    print(f"الدقة (R2): {metrics['R2']:.4f}")


In [11]:
metrics, y_pred = evaluate_model(best_rf_model,X_test,y_test)
print_metrics(metrics)

نتائج تقييم النموذج:
الدقة (MSE): 587327140.7422
الدقة (rmse): 24234.8332
الدقة (R2): 0.8161


In [13]:
joblib.dump(best_rf_model,OUTPUT_MODEL_PATH)

['../output/models/model.joblib']